<h1>IMDB Dataset Sentiment Analysis</h1>

We take the IMDB dataset to extract senitment from a movie which classifies either it is positive or negative

In [46]:

##IMporting the required Libraries

import string 
import numpy as np
import pandas as pd


In [47]:
##reading the contents of txt file into a list
with open("imdb_labelled.txt") as f:
    conntent=f.readlines()

In [48]:
conntent[:5] 

['A very, very, very slow-moving, aimless movie about a distressed, drifting young man.  \t0\n',
 'Not sure who was more lost - the flat characters or the audience, nearly half of whom walked out.  \t0\n',
 'Attempting artiness with black & white and clever camera angles, the movie disappointed - became even more ridiculous - as the acting was poor and the plot and lines almost non-existent.  \t0\n',
 'Very little music or anything to speak of.  \t0\n',
 'The best scene in the movie was when Gerardo is trying to find a song that keeps running through his head.  \t1\n']

In [49]:
##Removing the newline character from the list using .strip()
newcon=[]
for line in conntent:
    line=line.strip()## removing newline character
    newcon.append(line)
newcon[:6]

['A very, very, very slow-moving, aimless movie about a distressed, drifting young man.  \t0',
 'Not sure who was more lost - the flat characters or the audience, nearly half of whom walked out.  \t0',
 'Attempting artiness with black & white and clever camera angles, the movie disappointed - became even more ridiculous - as the acting was poor and the plot and lines almost non-existent.  \t0',
 'Very little music or anything to speak of.  \t0',
 'The best scene in the movie was when Gerardo is trying to find a song that keeps running through his head.  \t1',
 "The rest of the movie lacks art, charm, meaning... If it's about emptiness, it works I guess because it's empty.  \t0"]

In [50]:
"""splitting the sentiment values(0,1) from the sentence in list and 
            stroing it in seperate lists"""
review=[]
for x in newcon:
    x=x.split("\t")[0]
    review.append(x)##splitting review and adding to list
review[:5]

['A very, very, very slow-moving, aimless movie about a distressed, drifting young man.  ',
 'Not sure who was more lost - the flat characters or the audience, nearly half of whom walked out.  ',
 'Attempting artiness with black & white and clever camera angles, the movie disappointed - became even more ridiculous - as the acting was poor and the plot and lines almost non-existent.  ',
 'Very little music or anything to speak of.  ',
 'The best scene in the movie was when Gerardo is trying to find a song that keeps running through his head.  ']

In [51]:
label=[]
for x in newcon:
    x=x.split("\t")[1]
    label.append(x)## splitting labels and adding to list
label[:6]

['0', '0', '0', '0', '1', '0']

In [52]:
##Creating a dictionary to create a dataframe for pre-processing
connt={"review":review,"label":label}
df=pd.DataFrame(connt) ## creating data frame
print(df.shape)
df.head(10)

(1000, 2)


,review,label
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1
5,"The rest of the movie lacks art, charm, meanin...",0
6,Wasted two hours.,0
7,Saw the movie today and thought it was a good ...,1
8,A bit predictable.,0
9,Loved the casting of Jimmy Buffet as the scien...,1


<h3>Data Pre-Processing</h3>

In [55]:

#removing stopwords from each row of reviews using anonymous function
import nltk
from nltk.corpus import stopwords ##stopword removal
stop = stopwords.words('english')
df['review'] = df['review'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
print(stop[:10])
df.head(10)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]


,review,label
0,"A very, very, slow-moving, aimless movie distr...",0
1,"Not sure lost - flat characters audience, near...",0
2,Attempting artiness black & white clever camer...,0
3,Very little music anything speak of.,0
4,The best scene movie Gerardo trying find song ...,1
5,"The rest movie lacks art, charm, meaning... If...",0
6,Wasted two hours.,0
7,"Saw movie today thought good effort, good mess...",1
8,A bit predictable.,0
9,Loved casting Jimmy Buffet science teacher.,1


In [58]:
##removing punctuation
df['review'] = df['review'].apply(lambda x:''.join([i for i in x if i not in string.punctuation]))##punctuation removal
print(string.punctuation)
df.head(10)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


,review,label
0,A very very slowmoving aimless movie distresse...,0
1,Not sure lost flat characters audience nearly...,0
2,Attempting artiness black white clever camera...,0
3,Very little music anything speak of,0
4,The best scene movie Gerardo trying find song ...,1
5,The rest movie lacks art charm meaning If empt...,0
6,Wasted two hours,0
7,Saw movie today thought good effort good messa...,1
8,A bit predictable,0
9,Loved casting Jimmy Buffet science teacher,1


In [60]:
df["review"] = df["review"].apply(lambda x: x.lower())    ##converting to lower case
df.head(10)

,review,label
0,a very very slowmoving aimless movie distresse...,0
1,not sure lost flat characters audience nearly...,0
2,attempting artiness black white clever camera...,0
3,very little music anything speak of,0
4,the best scene movie gerardo trying find song ...,1
5,the rest movie lacks art charm meaning if empt...,0
6,wasted two hours,0
7,saw movie today thought good effort good messa...,1
8,a bit predictable,0
9,loved casting jimmy buffet science teacher,1


In [61]:
##using stemming to convert the words in review to its root stem
from nltk.tokenize import word_tokenize 
porter = nltk.PorterStemmer()
for i in range (df.shape[0]): 
  sents=df["review"].loc[i] ## every sentence taken from DF
  words=nltk.word_tokenize(sents)## every word is tokenized
  words=[porter.stem(word) for word in words] ## every word is stemmed
  df["review"].loc[i]=' '.join(words) ## word is joined back to the sentence

df.head()

,review,label
0,a veri veri slowmov aimless movi distress drif...,0
1,not sure lost flat charact audienc nearli half...,0
2,attempt arti black white clever camera angl mo...,0
3,veri littl music anyth speak of,0
4,the best scene movi gerardo tri find song keep...,1


<h3> Feature Extraction</h3>

In [62]:
## TF-IDF is used to vectorize the review statement holding the important words for the sentiment

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()

In [63]:
preview=df["review"].tolist()
ySet = label
xset_cv=tfidf.fit_transform(preview)
xset_cv

<1000x2580 sparse matrix of type '<class 'numpy.float64'>'
	with 8084 stored elements in Compressed Sparse Row format>

In [76]:
from sklearn.model_selection import train_test_split
xSet_train,xSet_test,ySet_train,ySet_test = train_test_split(xset_cv,ySet,test_size=0.2,random_state=45)## 80%training data

<h3>MODEL TRAINING AND VALIDATION</h3>

In [77]:
##Using Naive Bayes for training
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

NB = MultinomialNB()  
NB.fit(xSet_train,ySet_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [78]:

ySet_predict = NB.predict(xSet_test)

accuracyScore = accuracy_score(ySet_test,ySet_predict)*100

print("Prediction Accuracy :",accuracyScore) 


Prediction Accuracy : 78.0


In [90]:
temp=["  story was good worth a watch"]##fill your own sentence
tempo=tfidf.transform(temp)
XX1=NB.predict(tempo)
if XX1[0] == '0':
    print("negative ")
else:
    print("positive")

positive
